In [35]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
## Langmith tracking
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 7


In [36]:
import warnings

warnings.filterwarnings("ignore")
from pprint import pprint

from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language


In [37]:
loader = GenericLoader.from_filesystem(
    "./DotNetFramework/",
    glob="*",
    suffixes=[".cs"],
    parser=LanguageParser(language=Language.CSHARP, parser_threshold=1000),    
)
docs = loader.load()

In [38]:
docs[0].metadata['source']

'DotNetFramework/CustomWebControl.cs'

In [39]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

code_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.CSHARP, chunk_size=1000, chunk_overlap=0
)
documents = code_splitter.split_documents(docs)



In [40]:
len(documents)

3

In [41]:
## FAISS Vector Database
from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [42]:
# from langchain_community.llms import Ollama
# llm=Ollama(model="llama2")

from langchain_openai import ChatOpenAI
# llm=ChatOpenAI(model="gpt-3.5-turbo")
llm=ChatOpenAI(model="gpt-4-turbo")


In [43]:
# ## Design ChatPrompt Template
# from langchain_core.prompts import ChatPromptTemplate

# prompt=ChatPromptTemplate.from_messages(
#     [
#         ("system","Hi ChatGPT, we are going to rewrite some old C# code which is based on .NET Framework. please rewrite the piece of code with .NET 5/6+ or ASP.NET Core."),
#         ("user","old code:{oldcode}")
#     ]
# )



## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Hi ChatGPT, we are going to rewrite some old C# code which is based on .NET Framework. 
Please follow below instructions to design your new code:
(1) New Code should be based on .NET 8+ or ASP.NET Core. 
(2) Outmost namespace is AspNetCoreApp                                          
(3) Generate the Startup.cs file also
(4) Use Razor Pages page-based programming model to build web UI
(5) Use standard angle brackets (< and >) for generic types, don't use the HTML-encoded versions (&lt; and &gt;)    
                                                                       
Please follow below instructions to output response:
(1)Please use xml tag <description></description> to contain all your description; and use xml tag <NewCode><file name="xxxx"></file><file name="yyyy"></file>...</NewCode> to wrap source code files generated.
(2)The content within <file name="xxxx"></file> is valid C# code.
                                          
In addition, basedon the context provided, please help to identify.Net Framework features replaced. 

<context>
{context}
</context>
Old Code: {input}"""
)


In [44]:
## Chain Introduction
## Create Stuff Document Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [45]:
"""A retriever is an interface that returns documents given an unstructured query. 
It is more general than a vector store. A retriever does not need to be able to store documents,
only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, 
but there are other types of retrievers as well.
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f6dba11f150>)

In [46]:
from langchain.chains import create_retrieval_chain

retriever_chain  = create_retrieval_chain(retriever, document_chain)

In [47]:
# response=retriever_chain.invoke({"input": documents[0].page_content})
response=retriever_chain.invoke({"input": ""})

In [48]:
response['answer']

'<description>\nThe old code provided illustrates a typical .NET Framework WebForms application and a database query class using ADO.NET. The migration to .NET 8+ or ASP.NET Core involves several changes:\n\n1. Replacing WebForms with ASP.NET Core Razor Pages, which uses a page-based programming model similar to WebForms but with improved designs like better separation of concerns and testability.\n2. Replacing traditional ADO.NET code with Entity Framework Core for database operations, which provides a more robust and developer-friendly data access layer.\n3. Updating the project structure and configuration approach with a more modern and flexible system (e.g., using `Startup.cs` and dependency injection).\n\nThe provided solution will include a `Startup.cs` file and a Razor Page (replacing the WebForms Page and Custom Control), along with a data access class using Entity Framework Core.\n</description>\n<NewCode>\n    <file name="Startup.cs">\nusing Microsoft.AspNetCore.Builder;\nusi

In [49]:
import re
from IPython.display import display, Markdown

# Your input string
input_string = response['answer']



code_match = re.search(r'<NewCode>\n(.*?)\n</NewCode>', input_string, re.DOTALL)
description_match = re.search(r'<description>\n(.*?)\n</description>', input_string, re.DOTALL)

# Extracted code and description
code = code_match.group(1) if code_match else None
description = description_match.group(1) if description_match else None

print(code)


display(Markdown(description))



    <file name="Startup.cs">
using Microsoft.AspNetCore.Builder;
using Microsoft.AspNetCore.Hosting;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Hosting;

namespace AspNetCoreApp
{
    public class Startup
    {
        public void ConfigureServices(IServiceCollection services)
        {
            services.AddRazorPages();
        }

        public void Configure(IApplicationBuilder app, IWebHostEnvironment env)
        {
            if (env.IsDevelopment())
            {
                app.UseDeveloperExceptionPage();
            }
            else
            {
                app.UseExceptionHandler("/Error");
            }

            app.UseStaticFiles();

            app.UseRouting();

            app.UseAuthorization();

            app.UseEndpoints(endpoints =>
            {
                endpoints.MapRazorPages();
            });
        }
    }
}
    </file>
    <file name="Index.cshtml.cs">
using Microsoft.AspNetCore.Mvc.RazorPages;

name

The old code provided illustrates a typical .NET Framework WebForms application and a database query class using ADO.NET. The migration to .NET 8+ or ASP.NET Core involves several changes:

1. Replacing WebForms with ASP.NET Core Razor Pages, which uses a page-based programming model similar to WebForms but with improved designs like better separation of concerns and testability.
2. Replacing traditional ADO.NET code with Entity Framework Core for database operations, which provides a more robust and developer-friendly data access layer.
3. Updating the project structure and configuration approach with a more modern and flexible system (e.g., using `Startup.cs` and dependency injection).

The provided solution will include a `Startup.cs` file and a Razor Page (replacing the WebForms Page and Custom Control), along with a data access class using Entity Framework Core.

In [50]:
pattern = re.compile(r'<file name="(.+?)">(.*?)</file>', re.DOTALL)

# Finding all matches
matches = pattern.findall(code)

# Loop through matches and write to files
for file_name, file_content in matches:
    # Strip leading and trailing whitespaces from the content
    file_content = file_content.strip()
    # Open the file in write mode and write the content
    with open("./DotNet/"+file_name, 'w') as file:
        file.write(file_content)
    print(f"File saved: {file_name}")

File saved: Startup.cs
File saved: Index.cshtml.cs
File saved: Index.cshtml
File saved: DBManager.cs
